In [15]:
from __future__ import annotations
import pandas as pd
from pathlib import Path
from datetime import datetime
import os

In [16]:
BASE = Path("/Users/test/Desktop/phishing-detector")
INPUTS = [
    (BASE / "data/processed/legitimas/banca/legitimas_banca_limpio.csv", "banca"),
    (BASE / "data/processed/legitimas/cripto/legitimas_cripto_limpio.csv", "cripto"),
    (BASE / "data/processed/legitimas/ecommerce/legitimas_ecommerce_limpio.csv", "ecommerce"),
    (BASE / "data/processed/legitimas/energia/legitimas_energia_limpio.csv", "energia"),
]


OUTDIR = Path("data/processed")
OUTFILE = OUTDIR / "legitimas_consolidado_prototipo1.csv"
os.chdir("..")  
print("CWD =", Path.cwd())
# Verificación rápida de rutas (evita perder tiempo con typos)
missing = [p for p, _ in INPUTS if not Path(p).exists()]
print("Archivos faltantes:", missing if missing else "Ninguno")

CWD = /Users/test/Desktop/phishing-detector
Archivos faltantes: Ninguno


In [8]:
def load_one(path: str, sector: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"No existe: {p}")
    df = pd.read_csv(p)
    # Detectar columna URL (ajusta si en algún CSV se llama distinto)
    lower = {c.lower(): c for c in df.columns}
    if "url" in lower:
        url_col = lower["url"]
    elif "url_relevante" in lower:
        url_col = lower["url_relevante"]
        df = df.rename(columns={url_col: "url"})
        url_col = "url"
    else:
        raise ValueError(f"{p} sin columna URL. Columnas: {df.columns.tolist()}")

    # Normalización mínima
    df["url"] = df[url_col].astype(str).str.strip()
    df = df[df["url"].str.startswith(("http://", "https://"), na=False)]

    # Añadir trazabilidad
    df["sector"] = sector
    df["__source_file"] = p.name

    return df[["url", "sector", "__source_file"]]


In [10]:
# CELDA 3: prueba con el CSV de banca

test_path = (BASE / "data/processed/legitimas/banca/legitimas_banca_limpio.csv") 
df_banca = load_one(test_path, sector="banca")

print("Filas cargadas:", len(df_banca))
print("Columnas:", df_banca.columns.tolist())
df_banca.head(5)


Filas cargadas: 1111
Columnas: ['url', 'sector', '__source_file']


,url,sector,__source_file
0,https://www.caixabank.es/particular/home/parti...,banca,legitimas_banca_limpio.csv
1,https://www.caixabank.es/particular/general/co...,banca,legitimas_banca_limpio.csv
2,https://uniblog.unicajabanco.es,banca,legitimas_banca_limpio.csv
3,https://www.unicajabanco.es/es/particulares/cu...,banca,legitimas_banca_limpio.csv
4,https://www.unicajabanco.es/es/particulares/cu...,banca,legitimas_banca_limpio.csv


In [11]:
# CELDA 4: definir INPUTS y verificar que existen todas las rutas

INPUTS = [
    (BASE / "data/processed/legitimas/banca/legitimas_banca_limpio.csv", "banca"),
    (BASE / "data/processed/legitimas/cripto/legitimas_cripto_limpio.csv", "cripto"),
    (BASE / "data/processed/legitimas/ecommerce/legitimas_ecommerce_limpio.csv", "ecommerce"),
    (BASE / "data/processed/legitimas/energia/legitimas_energia_limpio.csv", "energia"),
]
faltan = [p for p, _ in INPUTS if not Path(p).exists()]
print("Archivos faltantes:", faltan if faltan else "Ninguno")

Archivos faltantes: Ninguno


In [12]:
# CELDA 5: consolidar datasets de todos los sectores

frames = []
for path, sector in INPUTS:
    df = load_one(path, sector)
    print(f"[OK] {sector}: {len(df)} filas")
    frames.append(df)

legit = pd.concat(frames, ignore_index=True)

# De-duplicado por (url, sector)
before = len(legit)
legit = legit.drop_duplicates(subset=["url", "sector"]).reset_index(drop=True)
removed = before - len(legit)

print(f"\nFilas totales tras consolidar: {len(legit)}")
print(f"Duplicados eliminados: {removed}")
legit.head()


[OK] banca: 1111 filas
[OK] cripto: 205 filas
[OK] ecommerce: 929 filas
[OK] energia: 817 filas

Filas totales tras consolidar: 3062
Duplicados eliminados: 0


,url,sector,__source_file
0,https://www.caixabank.es/particular/home/parti...,banca,legitimas_banca_limpio.csv
1,https://www.caixabank.es/particular/general/co...,banca,legitimas_banca_limpio.csv
2,https://uniblog.unicajabanco.es,banca,legitimas_banca_limpio.csv
3,https://www.unicajabanco.es/es/particulares/cu...,banca,legitimas_banca_limpio.csv
4,https://www.unicajabanco.es/es/particulares/cu...,banca,legitimas_banca_limpio.csv


In [13]:
OUTDIR = Path("data/processed"); OUTDIR.mkdir(parents=True, exist_ok=True)
OUTFILE = OUTDIR / "legitimas_consolidado.csv"

# añade timestamp de ingesta
legit["ts_ingesta"] = datetime.utcnow().isoformat(timespec="seconds") + "Z"

# checks mínimos (DoD)
assert {"url","sector","__source_file","ts_ingesta"}.issubset(set(legit.columns)), "Faltan columnas mínimas"
assert legit["url"].str.startswith(("http://","https://")).all(), "Hay URLs sin http(s)://"

# guardar
legit.to_csv(OUTFILE, index=False)

print(f"✅ Guardado: {OUTFILE}")
print(f"Filas: {len(legit)} | Columnas: {legit.columns.tolist()}")


✅ Guardado: data/processed/legitimas_consolidado.csv
Filas: 3062 | Columnas: ['url', 'sector', '__source_file', 'ts_ingesta']
